# Ensemble Mean
### Erica Simon, 02/20/24
## Purpose: calculate and visualize ensemble mean of temperature outcomes as generated by 3 different IAM emission trajectories
- calculates avg. temperature outcome for each config
- generates one distribution of 841 configs, rather than 3 x 841
- allows for more concise visualizations

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import seaborn as sns

## Globals

In [2]:
weights_51yr = np.ones(52)
weights_51yr[0] = 0.5
weights_51yr[-1] = 0.5

In [3]:
scenario_order = ["Net Zero 2050", "Low demand", "Below 2 C", "Delayed transition", 
                  "Fragmented World", "NDCs", "Current Policies"]

In [4]:
NGFS_colors = {
    "Net Zero 2050": "#00a9cf",
    "Below 2 C": "#003466",
    "NDCs": "#f69320",
    "Current Policies": "#df0000",
    "Low demand": "#2274ae",
    "Fragmented World": "#b0724e",
    "Delayed transition": "#92397a",
}

## Get datasets

In [5]:
# open temperature datasets
GCAM_t_ds = xr.open_dataset('~/outputs/GCAM_prob_run.nc')
MSG_t_ds = xr.open_dataset('~/outputs/MSG_prob_run.nc')
REM_t_ds = xr.open_dataset('~/outputs/REM_prob_run.nc')

## Calculate mean for each config

In [16]:
gcam = GCAM_t_ds.config.values
msg = MSG_t_ds.config.values
rem = REM_t_ds.config.values

print(np.setdiff1d(gcam, msg))
print(np.setdiff1d(msg, gcam))

print(np.setdiff1d(gcam, rem))
print(np.setdiff1d(rem, gcam))

print(np.setdiff1d(rem, msg))
print(np.setdiff1d(msg, rem))

[]
[]
[]
[]
[]
[]


In [ ]:
for config in GCAM_t_ds.config.values:
    c = GCAM_t_ds.where(GCAM_t_ds.config == 1234, drop=True)

In [27]:
GCAM_t_ds.where(GCAM_t_ds.config == 1234, drop=True)


<xarray.Dataset>
Dimensions:      (timebounds: 352, scenario: 7, config: 1, layer: 3)
Coordinates:
  * timebounds   (timebounds) float64 1.75e+03 1.751e+03 ... 2.1e+03 2.101e+03
  * scenario     (scenario) object 'Below 2 C' ... 'Net Zero 2050'
  * config       (config) int64 1234
  * layer        (layer) int64 0 1 2
Data variables:
    temperature  (timebounds, scenario, config, layer) float64 0.0 ... 0.9911